In [1]:
import sys, os, gc
import subprocess
from math import *
import numpy as np
from datetime import *
from pylab import *
import random
from datetime import datetime
import json
import imgaug as ia
import imgaug.augmenters as iaa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
def vgg6(input_shape=(128, 128, 3)):


    model = tf.keras.models.Sequential(name='VGG6')

    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, name='conv1'))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', name='conv2'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', name='conv3'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', name='conv4'))
    model.add(tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_2'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4)))
    model.add(tf.keras.layers.Dropout(0.25))
    
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', name='conv5'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', name='conv6'))
    model.add(tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_3'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4)))

    ## Fully connected layers
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(256, activation='relu', name='fc_2'))

       
    model.add(tf.keras.layers.Dense(128, activation='relu', name='fc_3'))
    
    model.add(tf.keras.layers.Dense(64, activation='relu', name='fc_4'))

    model.add(tf.keras.layers.Dense(8, activation='relu', name='fc_5'))
    
    # output layer
    model.add(tf.keras.layers.Dense(1, name='fc_out1'))

    return model

def esn_shuffle(array, seed=0):
        np.random.seed(seed)
        np.random.shuffle(array)
        return array

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


In [3]:
model = vgg6()
model.summary()

Model: "VGG6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 126, 126, 128)     3584      
_________________________________________________________________
conv2 (Conv2D)               (None, 124, 124, 128)     147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 62, 62, 128)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 60, 60, 256)       295168    
_________________________________________________________________
conv4 (Conv2D)               (None, 58, 58, 256)       590080    
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 58, 58, 256)       1024   

In [4]:
def esn_shuffle(array, seed=None):
        if seed is not None:
            np.random.seed(seed)
        np.random.shuffle(array)
        return array

In [5]:
seq = iaa.Sequential([
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.1))),
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 3), per_channel=0.5),
    iaa.Add((-5, 5), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    iaa.Sometimes(0.1, iaa.Grayscale(alpha=(0.0, 1.0))),
    iaa.Sometimes(0.50, iaa.Grayscale(alpha=1.)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, 
        mode=ia.ALL, cval=(0, 255))
    ], random_order=True)


seqGray = iaa.Sequential([
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, 
        mode=ia.ALL, cval=(255, 255))
    ], random_order=True)

In [6]:
%%time 

data = np.load('samples/RGB_128x128_test_000.npz')
images_RGB = data['images'].astype(np.uint8)
labels_RGB = data['labels'] 
pgcIDs_RGB = data['pgcIDs']
N = images_RGB.shape[0]
filter_RGB = np.chararray(N)
filter_RGB[:] = 'c'
data.close() 
del data


data = np.load('samples/g_128x128_test_000.npz')
images_g = data['images'].astype(np.uint8)
labels_g = data['labels'] 
pgcIDs_g = data['pgcIDs']
N = images_g.shape[0]
filter_g = np.chararray(N)
filter_g[:] = 'g'
data.close() 
del data

data = np.load('samples/r_128x128_test_000.npz')
images_r = data['images'].astype(np.uint8)
labels_r = data['labels'] 
pgcIDs_r = data['pgcIDs']
N = images_r.shape[0]
filter_r = np.chararray(N)
filter_r[:] = 'r'
data.close() 
del data

data = np.load('samples/i_128x128_test_000.npz')
images_i = data['images'].astype(np.uint8)
labels_i = data['labels'] 
pgcIDs_i = data['pgcIDs']
N = images_i.shape[0]
filter_i = np.chararray(N)
filter_i[:] = 'i'
data.close() 
del data

CPU times: user 2.35 s, sys: 97.9 ms, total: 2.45 s
Wall time: 2.44 s


In [7]:
images_gri = np.concatenate((images_g, images_r, images_i))
labels_gri = np.concatenate((labels_g, labels_r, labels_i))
pgcIDs_gri = np.concatenate((pgcIDs_g, pgcIDs_r, pgcIDs_i))
filter_gri = np.concatenate((filter_g, filter_r, filter_i))

N_RGB = len(labels_RGB)
N_gri = len(labels_gri)

In [8]:
ia.seed(100)

indx = esn_shuffle(np.arange(N_gri), seed=200)
images_aug = seqGray(images=images_gri[indx][:N_RGB,:,:,:])
labels_aug = labels_gri[indx][:N_RGB]
pgcIDs_aug = pgcIDs_gri[indx][:N_RGB]
filter_aug = filter_gri[indx][:N_RGB]

n = len(images_aug)
p = int(n/2)
images_aug[:p] = 255 - images_aug[:p]

ia.seed(200)

images_aug = np.concatenate((seq(images=images_RGB),images_aug))
labels_aug = np.concatenate((labels_RGB,labels_aug))
pgcIDs_aug = np.concatenate((pgcIDs_RGB,pgcIDs_aug))
filter_aug = np.concatenate((filter_RGB,filter_aug))

indx = np.arange(len(images_aug))
indx = esn_shuffle(indx, seed=100)
images_test_aug = images_aug[indx]
labels_test_aug = labels_aug[indx]
pgcIDs_test_aug = pgcIDs_aug[indx]
filter_test_aug = filter_aug[indx]

In [9]:
batches = esn_shuffle(np.arange(50), seed=50)

for i in range(1,60):
    batches = np.concatenate((batches, esn_shuffle(np.arange(50), seed=i*50))) 

batches, len(batches)

(array([34, 36,  1, ...,  2, 29,  7]), 3000)

In [10]:
zp_dir = 'Uset0_npz/'
ckpt_dir = 'U0_model03_ckpt/'

def trainer(iter, batches, suffix=None):

  if suffix is None:
    suffix = ''

  batchNo = batches[iter]
  if True: 

    vgg_model = vgg6()
    vgg_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse', 'mae'])

    if iter>0:

      vgg_model = vgg6()
      vgg_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse', 'mae'])
      vgg_model.load_weights(ckpt_dir+str(iter-1)+suffix+".ckpt")

      # if batchNo % 2 == 0 : 
      #   for layer in vgg_model.layers[:-1]:
      #     layer.trainable = False

      with open(ckpt_dir+'evalDict'+suffix+'.json') as json_file:
        evalDict = json.load(json_file)
    else:
      evalDict = {}


    print(iter, batchNo)


    batchFile = npzname = '128x128_train_aug_'+'%02d'%(batchNo+1)+'.npz'
    data = np.load(zp_dir + batchFile)
    images_train_aug = data['images'].astype(np.uint8)
    labels_train_aug = data['labels']
    pgcIDs_train_aug = data['pgcIDs']
    data.close() 
    del data



    n_epochs=1
    vgg_model.fit(images_train_aug, labels_train_aug, 
                                      epochs=n_epochs, batch_size=64, 
                                      validation_data=(images_test_aug, labels_test_aug),
                                      verbose=1, shuffle=True)
    evalDict[iter] = {} 
    for key in vgg_model.history.history:
      evalDict[iter][key] = vgg_model.history.history[key][0]
    evalDict[iter]["batchNo"] = batchNo
    evalDict[iter]["batchFile"] = batchFile
    
    with open(ckpt_dir+"evalDict"+suffix+".json", "w", encoding ='utf8') as outfile:
      json.dump(evalDict, outfile, allow_nan=True, cls=NpEncoder)
    
    vgg_model.save_weights(ckpt_dir+str(iter)+suffix+".ckpt")

    del vgg_model
    tf.keras.backend.clear_session()
    del images_train_aug 
    del labels_train_aug
    del pgcIDs_train_aug

    gc.collect()

    return evalDict

In [ ]:
with tf.device('/gpu:1'):
    for i in range(0, 2000):
        trainer(i, batches)

0 34
282/282 [==============================] - 195s 693ms/step - loss: 233.2632 - mse: 233.2632 - mae: 10.8256 - val_loss: 265.0927 - val_mse: 265.0927 - val_mae: 12.4368
1 36
282/282 [==============================] - 82s 290ms/step - loss: 99.2558 - mse: 99.2558 - mae: 7.9033 - val_loss: 394.5150 - val_mse: 394.5150 - val_mae: 17.7509
2 1
282/282 [==============================] - 74s 261ms/step - loss: 82.7708 - mse: 82.7708 - mae: 7.2129 - val_loss: 140.2159 - val_mse: 140.2159 - val_mae: 9.9014
3 38
282/282 [==============================] - 73s 259ms/step - loss: 69.4664 - mse: 69.4664 - mae: 6.5446 - val_loss: 114.7694 - val_mse: 114.7694 - val_mae: 8.5680
4 8
282/282 [==============================] - 73s 259ms/step - loss: 72.4480 - mse: 72.4480 - mae: 6.7016 - val_loss: 103.3370 - val_mse: 103.3370 - val_mae: 8.1159
5 16
282/282 [==============================] - 73s 259ms/step - loss: 62.0718 - mse: 62.0718 - mae: 6.1461 - val_loss: 192.8569 - val_mse: 192.8569 - val_mae: 1

282/282 [==============================] - 73s 259ms/step - loss: 23.9010 - mse: 23.9010 - mae: 3.6707 - val_loss: 26.9353 - val_mse: 26.9353 - val_mae: 3.8187
50 34
282/282 [==============================] - 73s 259ms/step - loss: 24.0278 - mse: 24.0278 - mae: 3.6515 - val_loss: 21.1430 - val_mse: 21.1430 - val_mae: 3.3696
51 36
282/282 [==============================] - 73s 259ms/step - loss: 23.3804 - mse: 23.3804 - mae: 3.6128 - val_loss: 20.3899 - val_mse: 20.3899 - val_mae: 3.3464
52 1
282/282 [==============================] - 73s 259ms/step - loss: 23.4282 - mse: 23.4282 - mae: 3.6124 - val_loss: 43.8831 - val_mse: 43.8831 - val_mae: 4.7980
53 38
282/282 [==============================] - 73s 259ms/step - loss: 23.0384 - mse: 23.0384 - mae: 3.5686 - val_loss: 19.5369 - val_mse: 19.5369 - val_mae: 3.1773
54 8
282/282 [==============================] - 73s 259ms/step - loss: 22.6653 - mse: 22.6653 - mae: 3.5639 - val_loss: 27.1530 - val_mse: 27.1530 - val_mae: 4.1437
55 16
282/28

282/282 [==============================] - 73s 259ms/step - loss: 17.4711 - mse: 17.4711 - mae: 3.0652 - val_loss: 18.4686 - val_mse: 18.4686 - val_mae: 3.0661
99 48
282/282 [==============================] - 73s 259ms/step - loss: 17.2385 - mse: 17.2385 - mae: 3.0963 - val_loss: 16.4176 - val_mse: 16.4176 - val_mae: 2.8416
100 6
282/282 [==============================] - 73s 259ms/step - loss: 15.9247 - mse: 15.9247 - mae: 2.9565 - val_loss: 17.5798 - val_mse: 17.5798 - val_mae: 2.9884
101 36
282/282 [==============================] - 73s 259ms/step - loss: 16.4723 - mse: 16.4723 - mae: 2.9883 - val_loss: 26.1416 - val_mse: 26.1416 - val_mae: 3.8415
102 37
282/282 [==============================] - 73s 259ms/step - loss: 16.8217 - mse: 16.8217 - mae: 3.0486 - val_loss: 18.4066 - val_mse: 18.4066 - val_mae: 3.1369
103 28
282/282 [==============================] - 73s 259ms/step - loss: 16.7487 - mse: 16.7487 - mae: 3.0213 - val_loss: 19.3545 - val_mse: 19.3545 - val_mae: 3.2780
104 43


282/282 [==============================] - 73s 259ms/step - loss: 15.1386 - mse: 15.1386 - mae: 2.8527 - val_loss: 22.6643 - val_mse: 22.6643 - val_mae: 3.4231
148 24
282/282 [==============================] - 73s 259ms/step - loss: 15.1391 - mse: 15.1391 - mae: 2.8442 - val_loss: 13.1787 - val_mse: 13.1787 - val_mae: 2.5988
149 8
282/282 [==============================] - 73s 259ms/step - loss: 14.8788 - mse: 14.8788 - mae: 2.8278 - val_loss: 19.9037 - val_mse: 19.9037 - val_mae: 3.2673
150 8
282/282 [==============================] - 73s 259ms/step - loss: 13.5090 - mse: 13.5090 - mae: 2.7302 - val_loss: 13.9217 - val_mse: 13.9217 - val_mae: 2.6766
151 31
282/282 [==============================] - 73s 258ms/step - loss: 14.3086 - mse: 14.3086 - mae: 2.7868 - val_loss: 15.2298 - val_mse: 15.2298 - val_mae: 2.8473
152 5
282/282 [==============================] - 73s 258ms/step - loss: 14.4867 - mse: 14.4867 - mae: 2.8089 - val_loss: 13.9588 - val_mse: 13.9588 - val_mae: 2.6939
153 2
28

282/282 [==============================] - 73s 259ms/step - loss: 13.0548 - mse: 13.0548 - mae: 2.6538 - val_loss: 15.4609 - val_mse: 15.4609 - val_mae: 2.8282
197 49
282/282 [==============================] - 73s 259ms/step - loss: 13.3666 - mse: 13.3666 - mae: 2.6925 - val_loss: 14.4681 - val_mse: 14.4681 - val_mae: 2.7355
198 25
282/282 [==============================] - 73s 259ms/step - loss: 13.7838 - mse: 13.7838 - mae: 2.7185 - val_loss: 15.4702 - val_mse: 15.4702 - val_mae: 2.9249
199 36
282/282 [==============================] - 73s 259ms/step - loss: 13.4767 - mse: 13.4767 - mae: 2.6918 - val_loss: 12.8825 - val_mse: 12.8825 - val_mae: 2.5101
200 33
282/282 [==============================] - 73s 259ms/step - loss: 12.7154 - mse: 12.7154 - mae: 2.6299 - val_loss: 14.7241 - val_mse: 14.7241 - val_mae: 2.8936
201 8
282/282 [==============================] - 73s 259ms/step - loss: 12.8047 - mse: 12.8047 - mae: 2.6349 - val_loss: 13.7655 - val_mse: 13.7655 - val_mae: 2.6321
202 31

...

641 37
282/282 [==============================] - 73s 258ms/step - loss: 8.6424 - mse: 8.6424 - mae: 2.1818 - val_loss: 15.4435 - val_mse: 15.4435 - val_mae: 2.8065
642 5
282/282 [==============================] - 73s 258ms/step - loss: 8.7104 - mse: 8.7104 - mae: 2.1963 - val_loss: 13.3468 - val_mse: 13.3468 - val_mae: 2.6118
643 33
282/282 [==============================] - 73s 258ms/step - loss: 8.6257 - mse: 8.6257 - mae: 2.1894 - val_loss: 12.6324 - val_mse: 12.6324 - val_mae: 2.4728
644 26
282/282 [==============================] - 73s 258ms/step - loss: 8.3816 - mse: 8.3816 - mae: 2.1459 - val_loss: 12.7390 - val_mse: 12.7390 - val_mae: 2.4915
645 10
282/282 [==============================] - 73s 258ms/step - loss: 8.5866 - mse: 8.5866 - mae: 2.1847 - val_loss: 12.1833 - val_mse: 12.1833 - val_mae: 2.4581
646 27
282/282 [==============================] - 73s 258ms/step - loss: 8.6834 - mse: 8.6834 - mae: 2.1908 - val_loss: 13.9687 - val_mse: 13.9687 - val_mae: 2.6520
647 48
282/

691 46
282/282 [==============================] - 73s 258ms/step - loss: 8.6133 - mse: 8.6133 - mae: 2.1699 - val_loss: 12.9800 - val_mse: 12.9800 - val_mae: 2.5417
692 6
282/282 [==============================] - 73s 258ms/step - loss: 8.0791 - mse: 8.0791 - mae: 2.1246 - val_loss: 11.9321 - val_mse: 11.9321 - val_mae: 2.4026
693 47
282/282 [==============================] - 73s 258ms/step - loss: 8.1957 - mse: 8.1957 - mae: 2.1310 - val_loss: 12.2932 - val_mse: 12.2932 - val_mae: 2.4306
694 22
282/282 [==============================] - 73s 258ms/step - loss: 9.1803 - mse: 9.1803 - mae: 2.2322 - val_loss: 12.2770 - val_mse: 12.2770 - val_mae: 2.4329
695 34
282/282 [==============================] - 73s 258ms/step - loss: 8.3278 - mse: 8.3278 - mae: 2.1522 - val_loss: 12.4778 - val_mse: 12.4778 - val_mae: 2.4441
696 4
282/282 [==============================] - 73s 258ms/step - loss: 8.2337 - mse: 8.2337 - mae: 2.1407 - val_loss: 11.6584 - val_mse: 11.6584 - val_mae: 2.4159
697 1
282/28

741 36
282/282 [==============================] - 73s 258ms/step - loss: 8.1271 - mse: 8.1271 - mae: 2.1235 - val_loss: 12.0632 - val_mse: 12.0632 - val_mae: 2.4125
742 41
282/282 [==============================] - 73s 258ms/step - loss: 7.8981 - mse: 7.8981 - mae: 2.1034 - val_loss: 12.7187 - val_mse: 12.7187 - val_mae: 2.5236
743 40
282/282 [==============================] - 73s 258ms/step - loss: 8.1603 - mse: 8.1603 - mae: 2.1183 - val_loss: 11.7730 - val_mse: 11.7730 - val_mae: 2.4158
744 16
282/282 [==============================] - 73s 258ms/step - loss: 7.9613 - mse: 7.9613 - mae: 2.1185 - val_loss: 11.7697 - val_mse: 11.7697 - val_mae: 2.4398
745 8
282/282 [==============================] - 73s 258ms/step - loss: 8.2969 - mse: 8.2969 - mae: 2.1339 - val_loss: 11.6032 - val_mse: 11.6032 - val_mae: 2.3933
746 28
282/282 [==============================] - 73s 259ms/step - loss: 8.1275 - mse: 8.1275 - mae: 2.1228 - val_loss: 12.2612 - val_mse: 12.2612 - val_mae: 2.4562
747 26
282/

791 8
282/282 [==============================] - 73s 258ms/step - loss: 7.8283 - mse: 7.8283 - mae: 2.0791 - val_loss: 11.5229 - val_mse: 11.5229 - val_mae: 2.3510
792 48
282/282 [==============================] - 73s 258ms/step - loss: 7.7459 - mse: 7.7459 - mae: 2.0711 - val_loss: 11.9774 - val_mse: 11.9774 - val_mae: 2.4072
793 1
282/282 [==============================] - 73s 258ms/step - loss: 7.9089 - mse: 7.9089 - mae: 2.1094 - val_loss: 11.9769 - val_mse: 11.9769 - val_mae: 2.4278
794 35
282/282 [==============================] - 73s 258ms/step - loss: 7.7151 - mse: 7.7151 - mae: 2.0789 - val_loss: 11.4359 - val_mse: 11.4359 - val_mae: 2.3548
795 39
282/282 [==============================] - 73s 258ms/step - loss: 7.7873 - mse: 7.7873 - mae: 2.0960 - val_loss: 12.5566 - val_mse: 12.5566 - val_mae: 2.4771
796 10
282/282 [==============================] - 73s 258ms/step - loss: 7.8701 - mse: 7.8701 - mae: 2.0891 - val_loss: 12.5357 - val_mse: 12.5357 - val_mae: 2.4849
797 36
282/2

841 18
282/282 [==============================] - 73s 258ms/step - loss: 8.1738 - mse: 8.1738 - mae: 2.1128 - val_loss: 11.5691 - val_mse: 11.5691 - val_mae: 2.3479
842 26
282/282 [==============================] - 73s 258ms/step - loss: 7.5443 - mse: 7.5443 - mae: 2.0517 - val_loss: 12.9619 - val_mse: 12.9619 - val_mae: 2.5144
843 46
282/282 [==============================] - 73s 258ms/step - loss: 7.7557 - mse: 7.7557 - mae: 2.0735 - val_loss: 12.8972 - val_mse: 12.8972 - val_mae: 2.5077
844 12
282/282 [==============================] - 73s 259ms/step - loss: 7.7973 - mse: 7.7973 - mae: 2.0763 - val_loss: 12.9985 - val_mse: 12.9985 - val_mae: 2.4929
845 49
282/282 [==============================] - 73s 258ms/step - loss: 7.6221 - mse: 7.6221 - mae: 2.0652 - val_loss: 12.1909 - val_mse: 12.1909 - val_mae: 2.4127
846 31
282/282 [==============================] - 73s 258ms/step - loss: 7.8001 - mse: 7.8001 - mae: 2.0822 - val_loss: 13.3958 - val_mse: 13.3958 - val_mae: 2.5500
847 25
282